In [ ]:
include("_.jl")

In [ ]:
using XMLDict

In [ ]:
xm = "msigdb_v7.5.1.xml"

In [ ]:
ca_se_di = Dict()

en = "GENESET"

for (id, li) in enumerate(readlines(joinpath(PAI, xm)))

    if !startswith(li, "<$en")

        continue

    end

    di = xml_dict(li)[en]

    ca = di[:CATEGORY_CODE]

    if ca == "ARCHIVED"

        continue

    end

    if !haskey(ca_se_di, ca)

        ca_se_di[ca] = Dict()

    end

    se_di = Dict(
        "Organism" => di[:ORGANISM],
        "Description" => di[:DESCRIPTION_BRIEF],
        "URL" => di[:EXTERNAL_DETAILS_URL],
        "ENSGs" => di[:MEMBERS],
        "Symbols" => di[:MEMBERS_SYMBOLIZED],
    )

    ca_se_di[ca][di[:STANDARD_NAME]] = se_di

end

OnePiece.dict.write(joinpath(PAO, replace(xm, ".xml" => ".json")), ca_se_di)

In [ ]:
co = joinpath(PAO, "collection")

mkpath(co)

for (ca, se_di) in ca_se_di

    se_ge_ = Dict(se => di["Symbols"] for (se, di) in se_di)

    pr = lowercase(ca)

    println("$pr has $(length(se_ge_)) sets.")

    OnePiece.dict.write(joinpath(co, "$pr.json"), se_ge_)

end

In [ ]:
for (ro, di_, fi_) in walkdir(PAI)

    for fi in fi_

        pr, ex = splitext(fi)

        if ex in [".gmt", ".tsv"]

            se_ge_ = OnePiece.gmt.read(joinpath(ro, fi))

            println("$pr has $(length(se_ge_)) sets.")

            OnePiece.dict.write(joinpath(PAO, "$pr.json"), se_ge_)

        end

    end

end